In [2]:
# !pip install openpyxl

In [3]:
!ls /data

Augmented_data.csv  Customer-Churn-Records.csv	bank_customer_churn_data.csv


In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("/data/Augmented_data.csv")
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())
df.head()

100%|██████████| 313067/313067 [00:00<00:00, 993755.75it/s] 


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15540053,Mears,296.190154,Spain,Male,20,5,0.000000,2,1,1,1.0,0,0,5,PLATINUM,1492.0,1-6-2022
1,15531365,Ellington,386.605052,Germany,Male,34,2,356279.021259,2,1,0,1.0,0,0,2,GOLD,748.0,1-2-2022
2,15537746,Handy,415.330857,Spain,Female,33,7,194086.893776,1,1,0,1.0,1,0,1,GOLD,1536.0,1-3-2021
3,15729553,Byrnes,204.893201,France,Female,34,8,0.000000,2,0,1,0.0,0,0,2,GOLD,2289.0,1-8-2022
4,15687688,Hou,340.349551,Germany,Female,32,10,205539.543083,2,1,0,1.0,1,0,1,SILVER,881.0,1-9-2020


In [ ]:
age_cache = {}
cust_id = list(df[CustomerId])
age = list(df["Age"])
tenure = list(df["Tenure"])
age = list(df["Age"])

In [5]:
cus_st = {}
for cus, st in zip(df["CustomerId"],df["Geography"]):
    if cus not in cus_st:
        cus_st[cus] = [st]
    else:
        if st not in cus_st[cus]:
            cus_st[cus].append(st)

In [6]:
df["Geography"] =  df["CustomerId"].progress_apply(lambda x: cus_st[x][0])

100%|██████████| 313067/313067 [00:00<00:00, 899005.47it/s]


In [7]:
cities = pd.read_excel("US_cities.xlsx")
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)
        
        
state_mapper = {}
for st in df["Geography"]:
    if st not in state_mapper:
        state_mapper[st] = np.random.choice(list(state_city.keys()))
        
city_mapper = {}
for cust_id, st in zip(df["CustomerId"],df["Geography"]):
    if cust_id not in city_mapper:
        city_mapper[cust_id] = np.random.choice(state_city[state_mapper[st]])

In [8]:
state_mapper

{'Spain': 'Vermont', 'Germany': 'Missouri', 'France': 'Michigan'}

In [9]:
df["Geography"] = df["Geography"].progress_apply(lambda x: state_mapper[x])
df["City"] = df["CustomerId"].progress_apply(lambda x: city_mapper[x])
df["month"] = df["date"].progress_apply(lambda x: x.split("-")[1])
df["year"] = df["date"].progress_apply(lambda x: x.split("-")[-1])
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: round(x,0))
df["Account_type"] = df["Tenure"].progress_apply(lambda x: "Gold" if x > 15 else "Premium" if x <= 15 and x >=8 else "Normal")

100%|██████████| 313067/313067 [00:00<00:00, 1212018.27it/s]


In [ ]:
age_cache = {}
cust_id = list(df[CustomerId])
age = list(df["Age"])
tenure = list(df["Tenure"])
year = list(df["year"])
month = list(df["month"])

for i, (c_id, ag, ten, y, mon) in enumerate(zip(cust_id, age, tenure, year, month)):
    if c_id not in age_cache:
        age_cache[cust_id] = {"month"}
    

In [8]:
df["Other Accounts"] = df["Tenure"].progress_apply(lambda x: "Yes" if x > 7 else "No")
df["Salary Account"] = df["Satisfaction Score"].progress_apply(lambda x: "Yes" if x >= 3 else "No" )
df["HasCrCard"] = df["HasCrCard"].progress_apply(lambda x: "Yes" if x == 1 else "No")
df["IsActiveMember"] = df["IsActiveMember"].progress_apply(lambda x: "Yes" if x == 1 else "No")
df["Exited"] = df["Exited"].progress_apply(lambda x: "Exited" if x == 1 else "Not Exited")
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: 350 if x <=350 else x)

100%|██████████| 283219/283219 [00:00<00:00, 1087420.25it/s]


In [9]:
def financial_year(x):
    mon = int(x[0])
    year = int(x[1])
    if year == 2020:
        if mon <=3:
            return "FY19-20"
        else:
            return "FY20-21"
    if year == 2021:
        if mon <=3:
            return "FY20-21"
        else:
            return "FY21-22"
    if year == 2022:
        if mon <=3:
            return "FY21-22"
        else:
            return "FY22-23"
        
    if year == 2023:
        if mon <=3:
            return "FY22-23"
        else:
            return "FY23-24"
    if year == 2024:
        if mon <=3:
            return "FY23-24"
        else:
            return "FY24-25"

df["Financial_year"] = df[["month", "year"]].progress_apply(financial_year, axis=1)

100%|██████████| 283219/283219 [00:01<00:00, 165834.67it/s]


In [10]:
def cs_category(x):
    if x > 800:
        return "Exceptional"
    if x > 740:
        return "Very Good"
    if x > 670:
        return "Good"
    if x > 580:
        return "Fair"
    else:
        return "poor"
df["CS_Category"] = df["CreditScore"].progress_apply(cs_category)

df["SS_Category"] = df["Satisfaction Score"].progress_apply(lambda x: "Poor" if x <3 else "Good" if x < 4 else "Very Good")

df["Quarter"] = df["month"].progress_apply(lambda x: "Q4" if int(x)<=3 else "Q1" if int(x)<=6 else "Q2" if int(x)<=9 else "Q3" )

month_dic = {'May':5, 'Oct':10, 'Nov':11, 'Feb':2, 'Apr':4, 'Jul':7, 'Jan':1, 'Sep':9, 'Jun':6,
       'Aug':8, 'Dec':12, 'Mar':3}
month_dic = {v:k for k,v in month_dic.items()}

df["month"] = df["month"].progress_apply(lambda x: month_dic[int(x)])

100%|██████████| 283219/283219 [00:00<00:00, 1072282.38it/s]


In [11]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,City,month,year,Account_type,Other Accounts,Salary Account,Financial_year,CS_Category,SS_Category,Quarter
0,15638257,P'an,635.0,Idaho,Female,54,0,9.433513e+04,2,Yes,...,Nampa,Mar,2020,Normal,No,No,FY19-20,Fair,Poor,Q4
1,15574676,Becerra,350.0,Missouri,Male,94,62,1.502475e+06,1,Yes,...,St. Joseph,Sep,2023,Gold,Yes,Yes,FY23-24,poor,Good,Q2
2,15578593,Schell,545.0,California,Male,47,21,2.540961e+05,1,Yes,...,Victorville,Mar,2022,Gold,Yes,No,FY21-22,poor,Poor,Q4
3,15876164,Christ,350.0,Idaho,Female,59,29,6.333844e+05,1,No,...,Coeur d'Alene,Apr,2022,Gold,Yes,Yes,FY22-23,poor,Very Good,Q1
4,15875372,Danielson,350.0,California,Female,80,49,7.462629e+05,1,Yes,...,Moreno Valley,May,2023,Gold,Yes,Yes,FY23-24,poor,Very Good,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283214,15610711,Eluemuno,547.0,California,Female,40,8,1.860525e+05,1,No,...,Napa,Jul,2020,Premium,Yes,No,FY20-21,poor,Poor,Q2
283215,15599809,Rousseau,796.0,Missouri,Male,29,1,4.480955e+04,1,Yes,...,Independence,Jan,2020,Normal,No,No,FY19-20,Very Good,Poor,Q4
283216,15947762,Dubois,350.0,Missouri,Male,139,102,0.000000e+00,2,Yes,...,St. Joseph,Sep,2024,Gold,Yes,Yes,FY24-25,poor,Very Good,Q2
283217,15451307,Benner,350.0,Idaho,Male,128,80,0.000000e+00,2,No,...,Coeur d'Alene,Mar,2024,Gold,Yes,Yes,FY23-24,poor,Very Good,Q4


In [12]:
df.columns

Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Complain', 'Satisfaction Score',
       'Card Type', 'Point Earned', 'date', 'City', 'month', 'year',
       'Account_type', 'Other Accounts', 'Salary Account', 'Financial_year',
       'CS_Category', 'SS_Category', 'Quarter'],
      dtype='object')

In [13]:
df.rename(columns={'Exited': 'Customer Status'}, inplace=True)

In [14]:
df.to_csv("/data/bank_customer_churn_data.csv", index=False)